### Optimization - QCQP, barrier method

##### 1.다음과 같은 QCQP 형태의 최적화 문제를 고려.
##### $\text{minimize}_x \quad \frac{1}{2}x^TQx + p^Tx $ 
##### $\text{s.t} \quad Ax = b, 
##### $\text{s.t} \quad \frac{1}{2}x^TQ_ix + p_{i}^{T}x \leq 0  \quad i = 1, \dots , m$

##### 위의 부등제약 조건을 아래와 같이 지시함수 형태로 포함, 여전히 boundary를 가지고 잇고, 미분 불가능 하기 때문에 계산상 어려움 존재 
##### $C = \{x:\frac{1}{2}x^TQ_ix + p_{i}^{T}x \leq 0 \quad i = 1, \dots , m\}$

##### $\text{minimize}_x \quad \frac{1}{2}x^TQx + p^Tx + I_c(x)$ 
##### 해결) barrier function을 이용해 boundary 포함하지않고, 미분가능 하게해서 newton's method 적용

##### 2.logarithmic barrier function 구하기
##### $h_i(x)=\frac{1}{2}x^TQ_ix + p_{i}^{T}x 라고 하자.$ -> convex이고, 두번 미분가능
##### $\phi(x)= -\sum_{i=1}^{m}log(-h_i(x))$ -> logarithmic barrier function

##### 3.barrier function을 적용 시킨 최적화 문제.
##### $\text{minimize}_x \quad t(\frac{1}{2}x^TQx + p^Tx) + \phi(x)$ 
##### $\text{s.t} \quad Ax = b$
##### 이와 같이 근사해서 정의된 문제를 newton's method로 푸는 방법을 barrier method라고 함.

In [45]:
import numpy as np
import matplotlib.pyplot as plt

In [60]:

def log_barrier(x):
    """ x에서 logarithmic barrier function
    """
    x = - np.sum(np.log(-x))
    return x


def function(x, Q, p, t):
    """ function = t(1/2*xTQx + pTx) + phi(x)
    Q, p : 쿼드라틱 폼의 메트릭스
    st. Ax = b
    """
    x = t * (1/2*np.dot(x.T, Q.dot(x)) + np.dot(p.T,x)) + log_barrier(1/2*np.dot(x.T, Q.dot(x)) + np.dot(p.T,x))
    return x



##### log barrier 계산.
##### $h_i(x)=\frac{1}{2}x^TQ_ix + p_{i}^{T}x$ 
##### $\phi(x)= -\sum_{i=1}^{m}log(-h_i(x))$ 
##### gradient: $\bigtriangledown \phi(x) = - \sum_{i=1}^{m} \frac{1}{h_i(x)}\bigtriangledown h_i(x)$
##### Hessian: $\bigtriangledown^2 \phi(x) = \sum_{i=1}^{m} \frac{1}{h_i(x)^2}\bigtriangledown h_i(x)\bigtriangledown h_i(x)^T - \sum_{i=1}^{m} \frac{1}{h_i(x)}\bigtriangledown^2 h_i(x) $

In [63]:

def grad(x, t, Q, p):
    """ function = t(1/2*xTQx + pTx) + phi(x) 의 gradient
    st. Ax = b
    """
    x = t*(Q.dot(x) + p) - np.sum(np.divide(Q.dot(x) + p, 1/2*np.dot(x.T, Q.dot(x)) + np.dot(p.T,x)), axis = 1)
    
    return x

def hess(x, t, Q, p):
    """ function = t(1/2*xTQx + pTx) + phi(x) 의 헤시안
    st. Ax = b
    """
    x = t*Q + np.sum(np.divide(np.dot(Q.dot(x) + p, (Q.dot(x) + p).T), (1/2*np.dot(x.T, Q.dot(x)) + np.dot(p.T,x))^2), axis = 1) - np.sum(np.divide(Q, 1/2*np.dot(x.T, Q.dot(x)) + np.dot(p.T,x)), axis = 1)

    return x


##### $min_x \quad tf(x) + \phi(x)$
##### $s.t \quad Ax = b$